In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
train_df = pd.read_csv('tap_fun_train.csv')
test_df = pd.read_csv('tap_fun_test.csv')

Wall time: 26.8 s


In [3]:
def cleaning(data):
    data['register_time'] = data.register_time.index
    return data

In [4]:
y = train_df.prediction_pay_price
train_df.drop(['prediction_pay_price'],axis=1,inplace = True)
train_df = cleaning(train_df)
test_df = cleaning(test_df)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(train_df)
test_scale = scaler.transform(test_df)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.1)

### Linear Regression

In [29]:
%%time
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train, y_train)
# preds = reg.predict(test_df)
y_pred = reg.predict(x_test)
print('RMSE: {}'.format(mean_squared_error(y_pred,y_test,multioutput = 'uniform_average')**0.5))

RMSE: 85.85807636297329
Wall time: 17.2 s


### Ridge Regression

In [ ]:
%%time
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
reg2 = Ridge(alpha=1,solver = 'sag')
reg2.fit(x_train, y_train)
# preds = reg2.predict(test_scale)
y_pred = reg2.predict(x_test)
print(reg2.score(x_train, y_train))
print('RMSE: {}'.format(mean_squared_error(y_pred,y_test,multioutput = 'uniform_average')**0.5))

In [48]:
%%time
from lightgbm import LGBMRegressor

reg3 = LGBMRegressor(n_estimators=700,n_jobs=-1)
reg3.fit(x_train, y_train)
# preds = reg2.predict(test_scale)
y_pred = reg3.predict(x_test)
print('RMSE: {}'.format(mean_squared_error(y_pred,y_test,multioutput = 'uniform_average')**0.5))

RMSE: 88.82965783768701
Wall time: 3min 42s


### Output Processing

In [9]:
## output processing
preds[preds<0.5] = 0
abb = pd.DataFrame(preds)
abb[(abb[0]>0) & (abb[0]<1)] = 0.99
sub = pd.DataFrame({'user_id':test_df.user_id,'prediction_pay_price': abb[0]},columns=['user_id','prediction_pay_price'])
sub.to_csv('output_processed_RF.csv',index=None)

In [10]:
sub.describe()

,user_id,prediction_pay_price
count,8.289340e+05,828934.000000
mean,1.811513e+06,2.880217
std,8.229500e+05,61.348869
min,1.493300e+04,0.000000
25%,1.286373e+06,0.000000
50%,2.012872e+06,0.000000
75%,2.481669e+06,0.000000
max,3.176543e+06,15790.062194


In [11]:
len(preds)

828934